In [1]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval

In [2]:
# Install package for PEP8 verification
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

Looking in indexes: https://test.pypi.org/simple/


In [3]:
#import pickle
#data = pickle.load(open('./data/StackOverflow_questions_2009_2020_cleaned_Bis2.csv', 'rb'))

#data = pd.read_csv("./data/StackOverflow_questions_2009_2020_cleaned_Bis2.csv",sep=";")


# Define path to data

path = './data/'
data = pd.read_csv(path+"StackOverflow_questions_cleaned_Bis2.csv",
                                   sep=";", index_col=0,
                   converters={"Title_c": literal_eval,
                               "Body_c": literal_eval,
                               #"Title": literal_eval,
                               #"Body": literal_eval,
                               "Tags": literal_eval})

 
data.head(3)

,Title_c,Body_c,Title,Body,Tags
Id,,,,,
59549222,"[android, gridlayout, java, program]","[code, button, button, row, space, button]",How to adjust Android gridlayout spacing from ...,I have the following code :\n \nIt looks like ...,[android]
59549258,"[discord, bot, user, data, file]","[bot, context, bot, youtuber, tutorial, algori...",My Discord XP bot isn't recognizing user data ...,My bot is not working as planned.\nFor the con...,[python]
59549286,"[way, user, selection, input, use]","[way, user, selection, input, example, app, ta...",A way for users to store selections as inputs ...,I'm trying to create a way in which a user can...,[r]


In [4]:
data.shape 

(26366, 5)

In [5]:
data["Doc_all"] = data["Title_c"] + data["Body_c"]
data["Doc_all_bis"] = data["Title"] + ' ' + data["Body"]
data["Doc_all"].head(3)
data["Doc_all_bis"].head(3)

Id
59549222    How to adjust Android gridlayout spacing from ...
59549258    My Discord XP bot isn't recognizing user data ...
59549286    A way for users to store selections as inputs ...
Name: Doc_all_bis, dtype: object

## Bag-of-words (via Tf-Idf)

La métrique TF-IDF (Term-Frequency - Inverse Document Frequency) utilise l'inverse de la fréquence du document comme indicateur de similarité, calculé sur une échelle logarithmique. Cette fréquence inverse est définie comme l'inverse de la proportion de documents qui contiennent le terme donné.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

# Définition de X et y
X = data["Doc_all"]
y = data["Tags"]

# Initialisation de TfidfVectorizer pour Full_doc
vectorizer = TfidfVectorizer(analyzer="word",
                             max_df=.6,
                             min_df=0.005,
                             tokenizer=None,
                             preprocessor=' '.join,
                             stop_words=None,
                             lowercase=False)

# Apprentissage du vectorizer sur les données X
vectorizer.fit(X)

# Transformation des données X en représentation TF-IDF
X_tfidf = vectorizer.transform(X)

print("Forme de X pour Doc_all : {}".format(X_tfidf.shape))

Forme de X pour Doc_all : (26366, 525)


Pour la préparation de nos cibles (pour les modèles supervisés), nous utiliserons la classe MultiLabelBinarizer de la bibliothèque Scikit-Learn. Cela est nécessaire car nos "Tags" sont multiples, ce qui signifie qu'un seul exemple peut être associé à plusieurs étiquettes.

In [7]:
# Binarisation multilabel pour les cibles y
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(y)
y_binarized = multilabel_binarizer.transform(y)

print("Forme de y : {}".format(y_binarized.shape))

Forme de y : (26366, 15)


In [8]:
from sklearn.model_selection import train_test_split

# Création des ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_binarized,
                                                    test_size=0.3, random_state=8)

print("Forme de X_train : {}".format(X_train.shape))
print("Forme de X_test : {}".format(X_test.shape))
print("Forme de y_train : {}".format(y_train.shape))
print("Forme de y_test : {}".format(y_test.shape))


Forme de X_train : (18456, 525)
Forme de X_test : (7910, 525)
Forme de y_train : (18456, 15)
Forme de y_test : (7910, 15)


##  Word2Vec 

Le principe de Word2Vec est une technique d'apprentissage automatique utilisée pour représenter les mots sous forme de vecteurs numériques denses dans un espace multidimensionnel. L'idée fondamentale derrière Word2Vec est que les mots ayant des significations similaires sont souvent utilisés dans des contextes similaires.

Word2Vec se base sur deux principaux modèles : CBOW (Continuous Bag-of-Words) et Skip-gram.

In [9]:
from gensim.models import Word2Vec

# Tokenisation des phrases
sentences = X

# Apprentissage du modèle Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Extraction des vecteurs de mots
word_vectors = model.wv

# Création de représentations vectorielles pour les phrases
X_w2v = np.array([
    np.mean([word_vectors[word] for word in sentence if word in word_vectors], axis=0)
    for sentence in sentences
])

print("Forme de X pour Word2Vec : {}".format(X_w2v.shape))

Forme de X pour Word2Vec : (26366, 100)


In [10]:
# Création des ensembles d'entraînement et de test
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y_binarized,
                                                    test_size=0.3, random_state=8)

print("Forme de X_train_w2v: {}".format(X_train_w2v.shape))
print("Forme de X_test_w2v : {}".format(X_test_w2v.shape))
print("Forme de y_train_w2v : {}".format(y_train_w2v.shape))
print("Forme de y_test_w2v : {}".format(y_test_w2v.shape))

Forme de X_train_w2v: (18456, 100)
Forme de X_test_w2v : (7910, 100)
Forme de y_train_w2v : (18456, 15)
Forme de y_test_w2v : (7910, 15)


## SBERT (Sentence-BERT)

Le principe de SBERT (Sentence-BERT) est une méthode d'apprentissage automatique utilisée pour représenter les phrases ou les phrases courtes sous forme de vecteurs numériques denses. SBERT est une extension de l'architecture BERT (Bidirectional Encoder Representations from Transformers) qui a été initialement conçue pour la représentation des mots.

SBERT utilise un modèle de langage pré-entraîné, comme BERT, qui est un réseau de neurones transformer capable de capturer les relations contextuelles entre les mots dans une phrase. Cependant, au lieu de se concentrer uniquement sur la prédiction du mot suivant dans une séquence, SBERT entraîne le modèle à effectuer des tâches de similarité sémantique entre les phrases.

In [11]:
#! pip install sentence-transformers

In [12]:
X[59549222]

['android',
 'gridlayout',
 'java',
 'program',
 'code',
 'button',
 'button',
 'row',
 'space',
 'button']

In [13]:
X_S = data["Doc_all_bis"]

In [14]:
X_S.values.tolist()

['How to adjust Android gridlayout spacing from Java program I have the following code :\n \nIt looks like this : \n\nIt only shows 22 x 8 buttons, I want to show 25 buttons in each row, and make the space between all buttons smaller, how to do it ?\n',
 "My Discord XP bot isn't recognizing user data in JSON files? My bot is not working as planned.\nFor the context of the bot, I followed a YouTuber's tutorial almost exactly (other than the algorithm for the experience) - https://www.youtube.com/watch?v=pKkrCHnun0M&t=890s\n \nI added the print function so I could track which functions have been triggered.\nThe first time, it worked. User data popped up in my users.json file when i typed in the channel. However, upon the second time typing anything, this happens:\n\nIt showed the same data being input twice! I was so confused because everything is the same. I will post the full code of the bot in here. Please help me make the bot possible to use.\nhttps://pastebin.com/BkzSbVan (I removed

In [17]:
!pip install sentence_transformers

In [18]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Extraction des représentations vectorielles pour les phrases
X_sbert = model.encode(X_S.values.tolist())
## X.tolist()[0]
###convert_to_tensor=True)
#X_sbert = model.encode(X, convert_to_tensor=True)

print("Forme de X pour SBERT : {}".format(X_sbert.shape))

KeyboardInterrupt: 

In [ ]:
X_sbert

In [ ]:
#!pip install tensorflow_hub

La méthode USE (Universal Sentence Encoder) est une approche utilisée dans le cadre de Natural Language Processing (NLP) pour la représentation sémantique des phrases. Elle est implémentée dans la bibliothèque nltk (Natural Language Toolkit) et utilise un modèle de réseaux de neurones pré-entraîné pour capturer les informations sémantiques et syntaxiques des phrases.

Le modèle USE est entraîné à partir de vastes quantités de données textuelles et apprend à représenter les phrases de manière à capturer leurs significations sémantiques. Il est capable de convertir une phrase en un vecteur numérique dense, appelé "embedding", qui représente les informations sémantiques de la phrase.

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub

# Chargement du modèle USE
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(model_url)

# Extraction des représentations vectorielles pour les phrases
#X_use = np.array(embed(X.tolist()))

X_use = model(X_S.values.tolist())


print("Forme de X pour USE : {}".format(X_use.shape))

## Modèle LDA, Latent Dirichlet Allocation

L'analyse latente de Dirichlet (LDA) est un modèle probabiliste utilisant deux valeurs de probabilité, à savoir la probabilité d'un mot donné étant donné les sujets (P(word|topics)) et la probabilité des sujets étant donné les documents (P(topics|documents)), pour obtenir des affectations de cluster. Initialement, ces valeurs sont calculées sur la base d'une attribution aléatoire, puis le calcul est répété pour chaque mot dans chaque document afin de décider de leur attribution à un sujet spécifique. Ce processus itératif permet de recalculer ces probabilités à plusieurs reprises jusqu'à ce que l'algorithme converge.

Nous allons utiliser la bibliothèque spécialisée Gensim pour entraîner un modèle unique basé sur la variable Doc_all. Dans cette partie, nous n'utiliserons pas le prétraitement TF-IDF, mais plutôt des fonctions spécifiques à Gensim.

Dans la première étape, nous allons créer le modèle Bag of Words ainsi que la matrice de fréquence des termes dans les documents :

In [ ]:
from gensim import corpora

# Create dictionnary (bag of words)
id2word = corpora.Dictionary(X)
id2word.filter_extremes(no_below=4, no_above=0.6, keep_n=None)
# Create Corpus 
texts = X  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

Gensim crée un identifiant unique pour chaque mot du document puis mappe word_id et word_frequency.

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

Nous allons à présent entrainer le modèle LDA sur Doc_all

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Construction du modèle LDA
full_lda_model = gensim.models.ldamulticore\
                    .LdaMulticore(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=20,
                                  random_state=8,
                                  per_word_topics=True,
                                  workers=4)
# Affichage du score de perplexité (qui est une mesure de l'adéquation du modèle aux données. 
# Une perplexité plus faible indique une meilleure performance du modèle)

print('\nPerplexité : ', full_lda_model.log_perplexity(corpus))

# Affichage du score de cohérence (La cohérence est une mesure de la qualité des thèmes générés par le modèle)
coherence_model_lda = CoherenceModel(model=full_lda_model,
                                     texts=texts,
                                     dictionary=id2word,
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nScore de cohérence : ', coherence_lda)


### Visualisation des résultats de LDA Gensim sur Doc_all 

La carte de distance interthèmes (Intertopic Distance Map en anglais) est une visualisation utilisée dans l'analyse de modèles de topic modeling, en particulier pour les modèles LDA (Latent Dirichlet Allocation). Elle est générée à l'aide d'une technique appelée multidimensional scaling (MDS) ou réduction multidimensionnelle.

La carte de distance interthèmes représente les relations de similarité ou de proximité entre les différents thèmes extraits par le modèle LDA. Chaque thème est représenté par un point ou un cercle sur la carte, et les distances entre les points reflètent la similarité entre les thèmes. Les thèmes similaires sont placés à proximité les uns des autres, tandis que les thèmes moins similaires sont éloignés.

In [ ]:
import pyLDAvis
from IPython.display import HTML
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
%matplotlib inline

display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))

gensimvis.prepare(full_lda_model, corpus, id2word)

 On calcule la matrice Document/Thème à l'aide de Gensim

In [ ]:
# Calculer la matrice Document/Thème avec Gensim
doc_topic = pd.DataFrame(full_lda_model.get_document_topics(corpus, minimum_probability=0))
for topic in doc_topic.columns:
    doc_topic[topic] = doc_topic[topic].apply(lambda x: x[1])
    
# Multiplication matricielle
topic_tag = np.matmul(doc_topic.T, y_binarized)
dimensions = topic_tag.shape

print('document/tag : ', y_binarized.shape)
print('document/thème : ', doc_topic.shape)
print('Dimensions de topic_tag : ', dimensions)

In [ ]:
doc_topic.head(3)

In [ ]:
topic_tag

Nous allons donc générer nos prédictions en sélectionnant les n premiers tags associés aux thèmes de chaque document :

In [ ]:
y_results = pd.DataFrame(y)
y_results["best_topic"] = doc_topic.idxmax(axis=1).values
y_results["nb_tags"] = y_results["Tags"].apply(lambda x : len(x))

df_y_bin = pd.DataFrame(y_binarized)
df_dict = dict(
    list(
        df_y_bin.groupby(df_y_bin.index)
    )
)

tags_num = []
for k, v in df_dict.items():
    check = v.columns[(v == 1).any()]
    tags_num.append(check.to_list())

y_results["y_true"] = tags_num
y_results.head(3)

In [ ]:
# Sélectionner les tags prédits dans la matrice des Topics / Tags
list_tag = []
for row in y_results.itertuples():
    nb_tags = row.nb_tags
    best_topic = row.best_topic
    row_tags = list(topic_tag.iloc[best_topic]\
                    .sort_values(ascending=False)[0:nb_tags].index)
    list_tag.append(row_tags)
    
y_results["y_pred"] = list_tag
y_results.head(3)

In [ ]:
def metrics_score(model, df, y_true, y_pred):
    """Fonction de compilation des métriques spécifiques aux problèmes de classification multi-étiquettes dans un DataFrame Pandas.
    Ce DataFrame aura 1 ligne par métrique et 1 colonne par modèle testé.

    Paramètres
    ----------------------------------------
    model : string
        Nom du modèle testé
    df : DataFrame
        DataFrame à étendre.
        Si None : Créer un DataFrame.
    y_true : array
        Tableau de valeurs réelles à tester
    y_pred : array
        Tableau de valeurs prédites à tester
    ----------------------------------------
    """
    if df is not None:
        temp_df = df
    else:
        temp_df = pd.DataFrame(index=["Précision", "F1",
                                      "Jaccard", "Rappel",
                                      "Précision"],
                               columns=[model])
        
    scores = []
    scores.append(metrics.accuracy_score(y_true, y_pred))
    scores.append(metrics.f1_score(y_pred, y_true, average='weighted'))
    scores.append(metrics.jaccard_score(y_true, y_pred, average='weighted'))
    scores.append(metrics.recall_score(y_true, y_pred, average='weighted'))
    scores.append(metrics.precision_score(y_true, y_pred, average='weighted'))
    temp_df[model] = scores
    
    return temp_df

In [ ]:
# Créer la matrice pour les prédictions y_pred et les valeurs réelles y_true de LDA
lda_y_pred = np.zeros(y_binarized.shape,dtype='uint8')
n = 0
for row in y_results.y_pred.values:
    for i in range(len(row)):
        lda_y_pred[n, row[i]] = 1
    n += 1
    
lda_y_true = np.zeros(y_binarized.shape,dtype='uint8')
m = 0
for row in y_results.y_true.values:
    for i in range(len(row)):
        lda_y_true[m, row[i]] = 1
    m += 1


In [ ]:
import sklearn.metrics as metrics

df_metrics_compare = metrics_score("LDA", df=None,
                                   y_true=lda_y_true,
                                   y_pred=lda_y_pred)
df_metrics_compare

Les résultats montrent clairement que la méthode LDA n'est pas adapté pour ce problème. 


### Modèle NMF

NMF (Non-Negative Matrix Factorization) est une technique d'apprentissage non supervisée utilisée pour la décomposition matricielle. Il est souvent utilisé dans le domaine du traitement du signal et de l'analyse de données pour extraire des informations cachées à partir de données non négatives.

In [ ]:
def plot_top_words(model, feature_names, 
                   n_top_words, nb_topic_plot, title):

#Paramètres
#----------------------------------------
#model : modèle NMF
#    Modèle NMF ajusté à afficher
#feature_names : tableau
#    Catégories résultantes du vectoriseur (TFIDF ...)
#n_top_words : entier
#    Nombre de mots pour chaque sujet.
#title : chaîne de caractères
#    Titre du graphique.
#----------------------------------------

    rows = int(nb_topic_plot/6)
    fig, axes = plt.subplots(rows, 6, figsize=(30, rows*10), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        if(topic_idx < nb_topic_plot):
            top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
            top_features = [feature_names[i] for i in top_features_ind]
            weights = topic[top_features_ind]

            ax = axes[topic_idx]
            bartopic = ax.barh(top_features, weights, height=0.7)
            bartopic[0].set_color('#f48023')
            ax.set_title(f'Sujet {topic_idx +1}', fontdict={'fontsize': 30})
            ax.invert_yaxis()
            ax.tick_params(axis='both', which='major', labelsize=20)
            for i in 'top right left'.split():
                ax.spines[i].set_visible(False)
            fig.suptitle(title, fontsize=36, color="#641E16")

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
import time
from sklearn.decomposition import NMF

# Définir le nombre de sujets à tester
best_nb_topics = 20
n_topics = best_nb_topics

print("-"*50)
print("Start NMF fitting on Doc_all ...")
print("-" * 50)
start_time = time.time()
# Initialisation de NMF
full_nmf = NMF(n_components=n_topics,
               init='nndsvd',
               random_state=8)

# Ajuster NMF sur le vecteur X_tfidf
full_nmf.fit(X_tfidf)

exec_time = time.time() - start_time
print("End of training :")
print("Execution time : {:.2f}s".format(exec_time))
print("-" * 50)

# Afficher les 12 premiers sujets
ff_feature_names = vectorizer.get_feature_names_out()
plot_top_words(full_nmf, ff_feature_names, 20, 12,
               'Sujets dans le modèle NMF pour Doc_all')

La modélisation avec NMF nous offre des catégories aussi compréhensibles que celles générées par l'algorithme LDA. Cependant, les thèmes générés restent très généraux et ne permettent pas une catégorisation cohérente pour notre problème d'auto-tagging. 

### Régression logistique avec OneVsRestClassifier

En utilisant la première méthode d'extraction de texte (TF_IDF)

In [ ]:
import time
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier

# Initialiser la régression logistique avec OneVsRest 
parametres_logit = {
"estimator__C": [100, 10, 1.0, 0.1],
"estimator__penalty": ["l1", "l2"],
"estimator__dual": [False],
"estimator__solver": ["liblinear"]
}

classificateur_logit = LogisticRegression()
# ExtraTreeclassifier 
one_vs_rest_logit = OneVsRestClassifier(classificateur_logit)

grid_search_logit = GridSearchCV(
one_vs_rest_logit,
param_grid=parametres_logit,
n_jobs=-1,
cv=5,
scoring="f1_weighted",
return_train_score=True,
refit=True
)

grid_search_logit.fit(X_train, y_train)

In [ ]:
logit_cv_results = pd.DataFrame.from_dict(grid_search_logit.cv_results_)
print("-"*50)
print("Best params for Logistic Regression")
print("-" * 50)
logit_best_params = grid_search_logit.best_params_
print(logit_best_params)

In [ ]:
logit_cv_results[logit_cv_results["params"]==logit_best_params]

Maintenant, nous pouvons effectuer des prédictions à l'aide du modèle de régression logistique sur le jeu test

In [ ]:
# Prediction
y_test_predicted_labels_tfidf = grid_search_logit.predict(X_test)

# transformation inverse 
y_test_pred_inversed = multilabel_binarizer\
    .inverse_transform(y_test_predicted_labels_tfidf)
y_test_inversed = multilabel_binarizer\
    .inverse_transform(y_test)

print("-"*50)
print("Affichage des 5 premières étiquettes prédites par rapport aux étiquettes réelles")
print("-" * 50)
print("Pred:", y_test_pred_inversed)
print("True:", y_test_inversed)

In [ ]:
df_metrics_compare = metrics_score("Logit", 
                                   df=df_metrics_compare, 
                                   y_true = y_test,
                                   y_pred = y_test_predicted_labels_tfidf)
df_metrics_compare

En utilisant la deuxième méthode d'extraction de texte (Word2Vec)

In [ ]:
import time
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier

# Initialiser la régression logistique avec OneVsRest 
parametres_logit_w2v = {
"estimator__C": [10],
"estimator__penalty": ["l1"],
"estimator__dual": [False],
"estimator__solver": ["liblinear"]
}

classificateur_logit = LogisticRegression()
one_vs_rest_logit = OneVsRestClassifier(classificateur_logit)

grid_search_logit = GridSearchCV(
one_vs_rest_logit,
param_grid=parametres_logit_w2v,
n_jobs=-1,
cv=5,
scoring="f1_weighted",
return_train_score=True,
refit=True
)

grid_search_logit.fit(X_train_w2v, y_train_w2v)

In [ ]:
logit_cv_results_w2v = pd.DataFrame.from_dict(grid_search_logit.cv_results_)
print("-"*50)
print("Best params for Logistic Regression")
print("-" * 50)
logit_best_params_w2v = grid_search_logit.best_params_
print(logit_best_params_w2v)

In [ ]:
logit_cv_results[logit_cv_results["params"]==logit_best_params_w2v]

In [ ]:
# Prediction
y_test_predicted_labels_w2v = grid_search_logit.predict(X_test_w2v)

# transformation inverse 
y_test_pred_inversed = multilabel_binarizer\
    .inverse_transform(y_test_predicted_labels_w2v)
y_test_inversed = multilabel_binarizer\
    .inverse_transform(y_test_w2v)

print("-"*50)
print("Affichage des 5 premières étiquettes prédites par rapport aux étiquettes réelles")
print("-" * 50)
print("Pred:", y_test_pred_inversed)
print("True:", y_test_inversed)

In [ ]:
df_metrics_compare = metrics_score("Logit avec w2v", 
                                   df=df_metrics_compare, 
                                   y_true = y_test_w2v,
                                   y_pred = y_test_predicted_labels_w2v)
df_metrics_compare